In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from nltk import sent_tokenize
from prepare import prepare

In [2]:
with open('data.json') as json_file:
    data = json.load(json_file)

In [3]:
df = pd.DataFrame(data)

In [41]:
df[df.language == 'Shell']

,repo,language,readme_contents,clean,stemmed,lemmatized
52,dockerfile/java,Shell,## Java Dockerfile\n\n\nThis repository contai...,java dockerfile repository contains dockerfile...,java dockerfil repositori contain dockerfil ja...,java dockerfile repository contains dockerfile...


# Check out prepare for prepare details

In [4]:
train, validate, test = prepare(df)

In [35]:
train.size, validate.size, test.size

(408, 180, 150)

In [36]:
train.head()

,repo,language,readme_contents,clean,stemmed,lemmatized
122,charlesmudy/responsive-html-email-template,HTML,Responsive HTML Email Template\n========\n\nTh...,responsive html email template template design...,respons html email templat templat design work...,responsive html email template template design...
104,coolwanglu/pdf2htmlEX,HTML,pdf2htmlEX is no longer under active developme...,pdf2htmlex longer active development new maint...,pdf2htmlex longer activ develop new maintain w...,pdf2htmlex longer active development new maint...
112,modularcode/modular-admin-html,HTML,# ModularAdmin: Free Bootstrap 4 Dashboard The...,modularadmin free bootstrap 4 dashboard theme ...,modularadmin free bootstrap 4 dashboard theme ...,modularadmin free bootstrap 4 dashboard theme ...
116,mailgun/transactional-email-templates,HTML,# Responsive transactional HTML email template...,responsive transactional html email templates ...,respons transact html email templat transact h...,responsive transactional html email template t...
7,freqtrade/freqtrade-strategies,Python,# Freqtrade strategies\n\nThis Git repo contai...,freqtrade strategies git repo contains free bu...,freqtrad strategi git repo contain free buysel...,freqtrade strategy git repo contains free buys...


# No duplicates

In [6]:
df[df.duplicated()]

,repo,language,readme_contents,clean,stemmed,lemmatized


# Exploration

In [7]:
def clean(text):
    'A simple function to cleanup text data'
    
    ADDITIONAL_STOPWORDS = []
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
    text = (unicodedata.normalize('NFKD', text)
             .encode('ascii', 'ignore')
             .decode('utf-8', 'ignore')
             .lower())
    words = re.sub(r'[^\w\s]', '', text).split()
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

In [8]:
def show_counts_and_ratios(df, column):
    """
    Takes in a dataframe and a string of a single column
    Returns a dataframe with absolute value counts and percentage value counts
    """
    labels = pd.concat([df[column].value_counts(),
                    df[column].value_counts(normalize=True)], axis=1)
    labels.columns = ['n', 'percent']
    labels
    return labels

show_counts_and_ratios(df, "language")

,n,percent
HTML,29,0.239669
C#,27,0.223140
Python,25,0.206612
JavaScript,23,0.190083
Java,15,0.123967
Shell,1,0.008264
Ruby,1,0.008264


In [9]:
python_words = clean(' '.join(df[df.language == 'Python'].readme_contents))

In [10]:
c_sharp_words = clean(' '.join(df[df.language == 'C#'].readme_contents))

In [11]:
html_words = clean(' '.join(df[df.language == 'HTML'].readme_contents))

In [12]:
javascript_words = clean(' '.join(df[df.language == 'JavaScript'].readme_contents))

In [13]:
java_words = clean(' '.join(df[df.language == 'Java'].readme_contents))

In [14]:
ruby_words = clean(' '.join(df[df.language == 'Ruby'].readme_contents))

In [15]:
shell_words = clean(' '.join(df[df.language == 'Shell'].readme_contents))

### Freq

In [16]:
python_freq = pd.Series(python_words).value_counts()

In [17]:
c_sharp_freq = pd.Series(c_sharp_words).value_counts()

In [18]:
html_freq = pd.Series(html_words).value_counts()

In [19]:
javascript_freq = pd.Series(javascript_words).value_counts()

In [20]:
java_freq = pd.Series(java_words).value_counts()

In [21]:
ruby_freq = pd.Series(ruby_words).value_counts()

In [22]:
shell_freq = pd.Series(shell_words).value_counts()

In [23]:
python_freq.head()

yes        1615
unknown     909
apikey      558
video       489
python      447
dtype: int64

In [24]:
c_sharp_freq.head()

new       192
c         188
use       186
var       185
string    156
dtype: int64

In [25]:
html_freq.head()

html        97
use         76
file        76
15001700    62
dom         51
dtype: int64

In [26]:
javascript_freq.head()

javascript    309
function      276
const         261
1             240
bad           205
dtype: int64

In [27]:
java_freq.head()

aligncentera    189
td              189
width100px      189
altbr           189
java            177
dtype: int64

### Ruby and Shell only have one observation so we can either drop or go find more

In [28]:
ruby_freq.head()

cookbook    6
chef        5
java        2
project     2
amazon      2
dtype: int64

In [29]:
shell_freq.head()

java              13
docker            10
run                7
jdk                5
dockerfilejava     5
dtype: int64